# Social Media Multi-Agent System

In this session, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
# pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew
# from crewai_tools import tool
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from crewai import Agent, Task, Crew, LLM

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file (contains API keys)
load_dotenv()

True

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
# import openai

# openai.api_key = " "

In [5]:
# import os
# #from utils import get_openai_api_key

# #openai_api_key = get_openai_api_key()
# import os
# os.environ["OPENAI_API_KEY"] = " "
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [6]:
# import os
# os.environ["SERPER_API_KEY"] = " "

In [7]:
llm = LLM(model="gpt-3.5-turbo")

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [8]:
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    llm=llm,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [11]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [12]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [13]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [14]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7c865add-2d8e-44c6-844e-f8bb4c648b19                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Thought: I need to start by researching the latest trends, key players, and noteworthy news on Artificial      │
│  Intelligence to prioritize the information based on relevance and importance. This will help in identifying    │
│  the target audience and crafting a detailed content outline including an introduction, key points, and a call  │
│  to action. Incorporating SEO keywords and relevant data or sources is crucial for maximizing the reach and     │
│  impact of the content. I will begin by searching the internet for the latest trends and key players in         │
│  Artificial Intelligence.                                                                                       │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

🚀 Crew: crew
├── 📋 Task: 39e62cb3-9361-4e82-990f-50c1796f7ec5
│   Assigned to: Content Planner
│   Status: ✅ Completed
│   ├── 🔧 Used Search the internet with Serper (1)
│   ├── 🔧 Using Read website content (1)
│   ├── 🔧 Using Search the internet with Serper (2)
│   └── 🔧 Used Read website content (2)
├── 📋 Task: a0ae06a8-0b1c-4b21-88fa-62c16501c0ad
│   Assigned to: Content Writer
│   Status: ✅ Completed
│   ├── 🔧 Used Read website content (3)
│   └── 🔧 Used Search the internet with Serper (3)
└── 📋 Task: 528f8745-6324-46cc-91a4-a72b97b61d23
    Status: Executing Task...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## The Evolution of Artificial Intelligence in 2021                                                            │
│                                                                                                                 │
│  Artificial Intelligence (AI) continues to reshape industries, redefine business operations, and elevate        │
│  customer experiences. In the landscape of AI for 2021, noteworthy trends are emerging that stand to alter our  │
│  perspectives on and utilization of this dynamic technology.                                                    │
│                                                                                                                 │
│  ### Ethical AI and Transparency                                                                                │
│  A prominent trend in 2021 is the heightened emphasis on ethical AI practices. With AI systems deeply embedded  │
│  in our daily routines, concerns regarding bias, privacy, and transparency have taken precedence. Companies     │
│  are now placing significant emphasis on integrating ethical considerations into AI development processes to    │
│  ensure fairness and accountability in the outcomes generated.                                                  │
│                                                                                                                 │
│  ### Advancements in Natural Language Processing (NLP)                                                          │
│  The year 2021 has witnessed remarkable progress in Natural Language Processing (NLP), empowering AI systems    │
│  to comprehend and produce human language with greater precision. This advancement is refining virtual          │
│  assistants, chatbots, and language translation services, thereby enriching communication experiences by        │
│  making them more seamless and efficient.                                                                       │
│                                                                                                                 │
│  ### AI-Powered Predictive Analytics                                                                            │
│  The widespread adoption of AI-driven predictive analytics across sectors such as finance, healthcare, and      │
│  marketing represents a significant trend. Through the utilization of AI algorithms to scrutinize extensive     │
│  datasets, organizations are now well-equipped to make data-driven forecasts concerning future trends,          │
│  customer behavior, and market dynamics. This capability leads to enhanced decision-making processes and        │
│  strategic planning.                                                                                            │
│                                                                                                                 │
│  ### Integrating AI with Augmented Reality (AR) and Virtual Reality (VR)                                        │
│  The convergence of AI with Augmented Reality (AR) and Virtual Reality (VR) is fostering immersive and          │
│  interactive encounters in various industries. Whether enhancing gaming simulations or revolutionizing          │
│  training initiatives, the amalgamation of AI with AR/VR is unveiling fresh avenues for user engagement and     │
│  innovation.                                           

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 528f8745-6324-46cc-91a4-a72b97b61d23                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7c865add-2d8e-44c6-844e-f8bb4c648b19                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## The Evolution of Artificial Intelligence in 2021                                              │
│                                                                                                                 │
│  Artificial Intelligence (AI) continues to reshape industries, redefine business operations, and elevate        │
│  customer experiences. In the landscape of AI for 2021, noteworthy trends are emerging that stand to alter our  │
│  perspectives on and utilization of this dynamic technology.                                                    │
│                                                                                                                 │
│  ### Ethical AI and Transparency                                                                                │
│  A prominent trend in 2021 is the heightened emphasis on ethical AI practices. With AI systems deeply embedded  │
│  in our daily routines, concerns regarding bias, privacy, and transparency have taken precedence. Companies     │
│  are now placing significant emphasis on integrating ethical considerations into AI development processes to    │
│  ensure fairness and accountability in the outcomes generated.                                                  │
│                                                                                                                 │
│  ### Advancements in Natural Language Processing (NLP)                                                          │
│  The year 2021 has witnessed remarkable progress in Natural Language Processing (NLP), empowering AI systems    │
│  to comprehend and produce human language with greater precision. This advancement is refining virtual          │
│  assistants, chatbots, and language translation services, thereby enriching communication experiences by        │
│  making them more seamless and efficient.                                                                       │
│                                                                                                                 │
│  ### AI-Powered Predictive Analytics                                                                            │
│  The widespread adoption of AI-driven predictive analytics across sectors such as finance, healthcare, and      │
│  marketing represents a significant trend. Through the utilization of AI algorithms to scrutinize extensive     │
│  datasets, organizations are now well-equipped to make data-driven forecasts concerning future trends,          │
│  customer behavior, and market dynamics. This capability leads to enhanced decision-making processes and        │
│  strategic planning.                                                                                            │
│                                                                                                                 │
│  ### Integrating AI with Augmented Reality (AR) and Virtual Reality (VR)                                        │
│  The convergence of AI with Augmented Reality (AR) and Virtual Reality (VR) is fostering immersive and          │
│  interactive encounters in various industries. Whether enhancing gaming simulations or revolutionizing          │
│  training initiatives, the amalgamation of AI with AR/

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the results of your execution as markdown in the notebook.

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

## The Evolution of Artificial Intelligence in 2021

Artificial Intelligence (AI) continues to reshape industries, redefine business operations, and elevate customer experiences. In the landscape of AI for 2021, noteworthy trends are emerging that stand to alter our perspectives on and utilization of this dynamic technology.

### Ethical AI and Transparency
A prominent trend in 2021 is the heightened emphasis on ethical AI practices. With AI systems deeply embedded in our daily routines, concerns regarding bias, privacy, and transparency have taken precedence. Companies are now placing significant emphasis on integrating ethical considerations into AI development processes to ensure fairness and accountability in the outcomes generated.

### Advancements in Natural Language Processing (NLP)
The year 2021 has witnessed remarkable progress in Natural Language Processing (NLP), empowering AI systems to comprehend and produce human language with greater precision. This advancement is refining virtual assistants, chatbots, and language translation services, thereby enriching communication experiences by making them more seamless and efficient.

### AI-Powered Predictive Analytics
The widespread adoption of AI-driven predictive analytics across sectors such as finance, healthcare, and marketing represents a significant trend. Through the utilization of AI algorithms to scrutinize extensive datasets, organizations are now well-equipped to make data-driven forecasts concerning future trends, customer behavior, and market dynamics. This capability leads to enhanced decision-making processes and strategic planning.

### Integrating AI with Augmented Reality (AR) and Virtual Reality (VR)
The convergence of AI with Augmented Reality (AR) and Virtual Reality (VR) is fostering immersive and interactive encounters in various industries. Whether enhancing gaming simulations or revolutionizing training initiatives, the amalgamation of AI with AR/VR is unveiling fresh avenues for user engagement and innovation.

### The Rise of Automated Detection and Prevention
Automated detection and prevention mechanisms fortified by AI have become indispensable in cybersecurity and fraud prevention realms. AI algorithms possess the agility to swiftly identify and counter potential threats, thereby reducing risks and fortifying the protection of sensitive data more effectively than conventional security measures.

In conclusion, the year 2021 is witnessing remarkable advancements in Artificial Intelligence, with trends such as ethical AI, NLP enhancements, predictive analytics, AI amalgamated with AR/VR, and automated security protocols taking the spotlight. Embracing the transformative capabilities of AI necessitates a thoughtful and responsible approach towards navigating these trends, paving the way for a future where AI augments human capabilities and propels progress.

In [ ]:
result